# QDOT SIMTOOL


In [ ]:
from hublib import use
from hublib.cmd import runCommand

In [ ]:
%use nemo-r19861
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

shape:
    type: Text
    value: cuboid
    description: Select the shape of the quantum dot.
    options: [cuboid, cylinder, dome, spheroid, cone, pyramid]

nstates:
    type: Number
    description: Number of eigen-energies to be solved for the system
    value: 8
    min: 1
    max: 20
  
dotx:
    type: Number
    description: X Dimensions
    value: 10
    min: 1
    max: 50
    units: nm
     
doty:
    type: Number
    description: Y Dimensions
    value: 10.5
    min: 1
    max: 50
    units: nm

dotz:
    type: Number
    description: Z Dimensions
    value: 5
    min: 1
    max: 50
    units: nm

lattice:
    type: Number
    description: Lattice Constant
    value: 0.565
    min: 0.3
    max: 10
    units: nm
        
emass:
    type: Number
    description: Effective Mass
    value: 0.067
    min: 0.005
    max: 3.0

Eg:
    type: Number
    description: Energy gap
    value: 1.43
    min: 0
    max: 20
    units: eV

gamma:
    type: Number
    description: State Broadening
    value: 0.01
    min: 0.00001
    max: 0.1

temperature:
    type: Number
    description: Ambient temperature of the quantum dot.
    value: 300
    min: 1
    max: 1000
    units: K

Ef:
    type: Number
    description: Electron Fermi Level
    value: 0
    min: -10
    max: 10
    units: eV
    
sweep:
    type: Number
    value: angle phi
    description: Sweep Parameter.
    options: [fermi energy, temperature, angle theta, angle phi]

smin:
    type: Number
    value: 0
    description: Minimum swept.
    min: 0
    max: 90

smax:
    type: Number
    value: 90
    description: Maximum swept.
    min: 0
    max: 180

spts:
    type: Number
    value: 3
    description: Number of Points.
    min: 2
    max: 30

phi:
    type: Number
    value: 0
    description: Angle Phi
    min: 0
    max: 360
    units: deg
    
theta:
    type: Number
    value: 45
    description: Angle Theta
    min: 0
    max: 360
    units: deg
    

In [ ]:
shape = "cuboid"
nstates = 8
dotx = 10
doty = 10.5
dotz = 5
lattice = 0.565
emass = 3.0
Eg = 1.43
gamma = 0.01
temperature = 300
Ef = 0
sweep = "angle theta"
smin = 0
smax = 90
spts = 3
phi = 0
theta = 45

In [ ]:
import sys
sys.path.insert(0, '/apps/simtools/qdot')
import qdot_helper
import numpy as np
from string import Template
from hublib.tool import save
NEMO_DB = '/apps/share64/debian7/nemo/r19861/mat/all.mat'

In [ ]:
#calculated parameters
import math
xn = math.ceil(dotx/lattice)
yn = math.ceil(doty/lattice)
zn = math.ceil(dotz/lattice)
T = -1.112e-68/(2*math.pow((lattice*1e-9),2)*9.11e-31*emass*1.6e-19)
E0 = -6*T + Eg
num_atoms = (dotx*doty*dotz)/math.pow(lattice,3)
cores, time = qdot_helper.eval_core_simtime(num_atoms, "VBCB")
shape_key = ['','','','','','']
shape_val = ['','','','','','']
if (cores == 1):
    matpath = NEMO_DB
else:
    matpath = 'all.mat'
if (shape == "cone"):
    height = dotz
    xcenter = dotx/2.0
    ycenter = doty/2.0
    radius1 = dotx/2.0
    shape_key[0] = "height ="
    shape_val[0] = height 
    shape_key[1] = "x_center ="
    shape_val[1] = xcenter 
    shape_key[2] = "y_center ="
    shape_val[2] = ycenter 
    shape_key[3] = "z_center ="
    shape_val[3] = 0 
    shape_key[4] = "radius1 ="
    shape_val[4] = radius1 
    shape_key[5] = "radius2 ="
    shape_val[5] = 0 
else :
    shape_key[0] = "min ="
    shape_val[0] = (0,0,0) 
    shape_key[1] = "max ="
    shape_val[1] = "("+str(dotx)+","+str(doty)+","+str(dotz)+")"


theta_vector = [theta for i in range(spts)]
phi_vector = [phi for i in range(spts)]
temp_vector = [temperature]
ef_vector = [Ef]
sweep_parameter = ""
sweep_values = spts
if sweep == "temperature":
    temp_vector = np.linspace(smin,smax,spts).tolist()
    sweep_parameter = "temp"
elif sweep == "fermi energy":
    ef_vector = np.linspace(smin,smax,spts).tolist()
    sweep_parameter = "ef"
elif sweep == "angle theta":
    theta_vector = np.linspace(smin,smax,spts).tolist()
    sweep_parameter = "angle"
    sweep_values = spts + 4
elif sweep == "angle phi":
    phi_vector = np.linspace(smin,smax,spts).tolist()
    sweep_parameter = "angle"
    sweep_values = spts + 4 
phi_vector_str = ','.join([str(num) for num in phi_vector])
theta_vector_str = ','.join([str(num) for num in theta_vector])
temp_vector_str = ','.join([str(num) for num in temp_vector])
ef_vector_str = ','.join([str(num) for num in ef_vector])
absoluteEf = False # TODO

In [ ]:
# put inputs in a dict for easy substitution with python format
inputs = {
    'nemo5shape': shape,
    'NSTATES': nstates,
    'dotx': dotx,
    'doty': doty,
    'dotz': dotz,
    'lattice': lattice,
    'emass': emass,
    'Eg': Eg,
    'gamma' : gamma,
    'temperature' : temperature,
    'Ef' : Ef,
    'sweep' : sweep,
    'smin' : min,
    'smax' : max,
    'spts' : spts,
    'phi' : phi,
    'theta' : theta,

    'xn' : xn,
    'yn' : yn,
    'zn' : zn,
    'T' : T,
    'E0' : E0,
    'num_atoms' : num_atoms,
    'cores' : cores,
    'time' : time,
    'matpath' : matpath,
    'shape_key1' : shape_key[0],
    'shape_val1' : shape_val[0],
    'shape_key2' : shape_key[1],
    'shape_val2' : shape_val[1],
    'shape_key3' : shape_key[2],
    'shape_val3' : shape_val[2],
    'shape_key4' : shape_key[3],
    'shape_val4' : shape_val[3],
    'shape_key5' : shape_key[4],
    'shape_val5' : shape_val[4],
    'shape_key6' : shape_key[5],
    'shape_val6' : shape_val[5],
    'phi_vector' : phi_vector_str,
    'theta_vector' : theta_vector_str,
    'temp_vector' : temp_vector_str,
    'ef_vector' : ef_vector_str,
    'sweep_values' : sweep_values,
    'sweep_parameter' : sweep_parameter,

}


In [ ]:
#Build Nemo input file
inputfile_name = 'input.in'
logname = 'log.txt'

inputfile = Template('''
Structure 
{ 
    Material 
    { 
        name               = GaAs 
        tag                = substrate 
        crystal_structure  = simplecubic 
        atoms              = (GaAs) 
        Lattice:a_lattice  = ${lattice}
        regions            = (1) 
        Bands:TB:s:param_set = nanohub 
        Bands:TB:s:nanohub:E_S_GaAs = ${E0}
        Bands:TB:s:nanohub:V_S_S_Sigma_GaAs_GaAs = ${T}
        Bands:TB:s:nanohub:V_S_S_Sigma_GaAs_H = -20 
        Bands:TB:s:nanohub:passivation_potential = 125 
    } 
   		
    Domain 
    { 
        name                   = structure1 
        type                   = pseudomorphic 
        base_material          = substrate 
        dimension              = (${xn},${yn},${zn}) 
        periodic               = (false, false, false)	
        crystal_direction1     = (1,0,0) 
        crystal_direction2     = (0,1,0) 
        crystal_direction3     = (0,0,1) 
        space_orientation_dir1 =  (1,0,0) 
        space_orientation_dir2 =  (0,1,0) 
        regions                = (1) 
        geometry_description   = simple_shapes 
    } 
   
    Geometry 
    { 
        Region 
        { 
            shape         = ${nemo5shape}
            region_number = 1 
            priority      = 1 
            tag           = substrate
            ${shape_key1} ${shape_val1}
            ${shape_key2} ${shape_val2}
            ${shape_key3} ${shape_val3}
            ${shape_key4} ${shape_val4}
            ${shape_key5} ${shape_val5}
            ${shape_key6} ${shape_val6}
        } 
    } 
} 
   
Solvers 
{ 
   
    solver 
    { 	
        name           = structure_out 
        type           = Structure 
        domain         = structure1 
        active_atoms_only = true 
        structure_file = shape.vtk 
        unit_cell_file = unit_cell.vtk 
        output_format  = vtk 
    } 
   
    solver 
    { 
        name           = surface_qd 
        type           = GeometryVIS 
        active_regions = (1) 
        geometry_surfaces_output = true 
        geometry_surfaces_resolution = 50 
        geometry_surfaces_filename = surface_qd.vtk 
    } 

    solver 
    { 
        name           = qm_states 
        type           = Schroedinger 
        max_number_iterations = 1000 
        convergence_limit = 1e-14  
        shift          = ${Eg} 
        solver_transformation_type = sinvert 
        domain         = structure1 
        active_regions = (1) 
        tb_basis       = s 
        job_list = (passivate_H, calculate_band_structure) 
        eigen_values_solver   = krylovschur 
        number_of_eigenvalues = ${NSTATES}
        monitor_convergence   = true 
        output          = (energies, eigenfunctions_VTP) 
        k_space_basis   = cartesian 
        k_points        = [(0, 0, 0)] 
        number_of_nodes = (1) 
    } 
   
    solver 
       { 
        name           = optical 
        type           = MatrixElements 
        domain         = structure1 
        active_regions = (1) 
        operator       = optical 
        wf_simulation  = qm_states 
        gamma = ${gamma}
        temp = (${temp_vector}) 
        ef = (${ef_vector}) 
        sweep = ${sweep_parameter}
        sweep_values = ${sweep_values}
        absorption   = true 
        transition   = true 
        egrid_size   = 1000 
        output_file  = matrix_elements 
        polarization_phis   = ($phi_vector   ,  0, 90, 0, $phi) 
        polarization_thetas = ($theta_vector , 90, 90, 0, $theta) 
    } 
   
} 
   
Global 
{ 
    solve = (structure_out, surface_qd, qm_states, optical ) 
    database = all.mat 
    messaging_level = 5 
    database = ${matpath}
    validate_options = false 
}''')



with open(inputfile_name, "w") as f:
    f.write(inputfile.safe_substitute(inputs))

In [ ]:
if (cores == 1):
    exitStatus,stdOutput,stdError = runCommand("nemo " + inputfile_name)
else:
    exitStatus,stdOutput,stdError = runCommand("submit -i " + NEMO_DB + " -n " + cores + " -w " + time + " nemo-r19861 " + inputfile_name)


In [ ]:
if cores == 1:
    fields = qdot_helper.resample_vtk_serial(["eigenfunctions/qm_states_f" + str(i)+ ".vtk" for i in range(nstates)], [30, 30, 30], 0.1, 'max', None)
else:
    fields = [] #TODO
evalues = np.array(fields)
save('Eigenfunctions', evalues)

In [ ]:
if cores == 1:
    fields = qdot_helper.resample_vtk_serial(["surface_qd.vtk" for i in range(nstates)], [30, 30, 30], 0.05, 'sum', -5)
else:
    fields = [] #TODO
surface = np.array(fields)
save('Surfaces', surface[0]) 

In [ ]:
estates = np.genfromtxt('qm_states_energies.dat', delimiter="\t", usecols=range(nstates))
save('Energy states', estates)

In [ ]:
transitionsX = np.genfromtxt('transitions_phi0_theta90.dat', delimiter=" ", usecols=range(2), skip_header=1)
transitionsX[:,1] = transitionsX[:,1] + 1e-30
save('X-polarized', transitionsX)

In [ ]:
transitionsY = np.genfromtxt('transitions_phi90_theta90.dat', delimiter=" ", usecols=range(2), skip_header=1)
transitionsY[:,1] = transitionsY[:,1] + 1e-30
save('Y-polarized', transitionsY)

In [ ]:
transitionsZ = np.genfromtxt('transitions_phi0_theta0.dat', delimiter=" ", usecols=range(2), skip_header=1)
transitionsZ[:,1] = transitionsZ[:,1] + 1e-30
save('Z-polarized', transitionsZ)

In [ ]:
transitionsAng = np.genfromtxt("transitions_phi" + str(phi) + "_theta" + str(theta) + ".dat", delimiter=" ", usecols=range(2), skip_header=1)
transitionsAng[:,1] = transitionsAng[:,1] + 1e-30
save('Angle-polarized', transitionsAng)

In [ ]:
Efoutput = "{0:g}".format(estates[0])
if (absoluteEf):
    Efoutput = "{0:g}".format(Ef)
if (sweep == "temperature"):
    filename = "absorption_phi_" + "{0:g}".format(phi) + "_theta_" + "{0:g}".format(theta) + "_temp_" + "{0:g}".format(temp_vector[0]) + "_ef_" + Efoutput + ".dat"
elif (sweep == "fermi energy"):
    Efoutput = "{0:g}".format(ef_vector[0]+estates[0])
    if (absoluteEf):
        Efoutput = "{0:g}".format(ef_vector[0])
    filename = "absorption_phi_" + "{0:g}".format(phi) + "_theta_" + "{0:g}".format(theta) + "_temp_" + "{0:g}".format(temperature) + "_ef_" + Efoutput + ".dat"
else:
    filename = "absorption_phi_" + "{0:g}".format(phi) + "_theta_" + "{0:g}".format(theta) + "_temp_" + "{0:g}".format(temperature) + "_ef_"  + Efoutput + ".dat"

absorption = np.genfromtxt(filename, delimiter=" ", usecols=range(2), skip_header=1)
absorption[:,1] = absorption[:,1] + 1e-30
save('Absorption', absorption)

In [ ]:
points = spts
absorption_list = []
for s in range(points):
    Efoutput = "{0:g}".format(estates[0])
    if (absoluteEf):
        Efoutput = "{0:g}".format(Ef)
    if (sweep == "temperature"):
        filename = "absorption_phi_" + "{0:g}".format(phi_vector[s]) + "_theta_" + "{0:g}".format(theta_vector[s]) + "_temp_" + "{0:g}".format(temp_vector[s]) + "_ef_" + Efoutput + ".dat"
    elif (sweep == "fermi energy"):
        Efoutput = "{0:g}".format(ef_vector[s]+estates[0])
        if (absoluteEf):
            Efoutput = "{0:g}".format(ef_vector[s])
        filename = "absorption_phi_" + "{0:g}".format(phi_vector[s]) + "_theta_" + "{0:g}".format(theta_vector[s]) + "_temp_" + "{0:g}".format(temperature) + "_ef_" + Efoutput + ".dat"
    else:
        filename = "absorption_phi_" + "{0:g}".format(phi_vector[s]) + "_theta_" + "{0:g}".format(theta_vector[s]) + "_temp_" + "{0:g}".format(temperature) + "_ef_"  + Efoutput + ".dat"

    absorption = np.genfromtxt(filename, delimiter=" ", usecols=range(2), skip_header=1)
    absorption[:,1] = absorption[:,1] + 1e-30    
    absorption_list.append(absorption)
save('Absorption Sweep', np.array(absorption_list))

In [ ]:
if (sweep == "temperature" or sweep == "fermi energy"):
    integrated_absortion = np.genfromtxt('integrated_absorption.dat', delimiter=" ", usecols=range(3), skip_header=1)
    integrated_absortion = np.delete(integrated_absortion, 2, 1)
elif (sweep == "angle phi"):
    integrated_absortion = np.genfromtxt('integrated_absorption.dat', delimiter=" ", usecols=range(3), skip_header=1, skip_footer=4)    
    integrated_absortion = np.delete(integrated_absortion, 1, 1)
else:
    integrated_absortion = np.genfromtxt('integrated_absorption.dat', delimiter=" ", usecols=range(3), skip_header=1, skip_footer=4)    
    integrated_absortion = np.delete(integrated_absortion, 0, 1)
save('Integrated Absortion', np.array(integrated_absortion))

In [ ]:
input_deck = ""
with open(inputfile_name, 'r') as input_text:
    input_deck = input_text.read()
save('input_deck', input_deck)

In [ ]:
save('Output Log', stdOutput.decode() + '-----------------\n' + stdError.decode())